Chargement des bibliothèques

In [2]:
import pandas as pd

Téléchargement et chargement des données

In [3]:
train_data = pd.read_csv("/kaggle/input/ghana-crop-disease/Train.csv")
test_data = pd.read_csv("/kaggle/input/ghana-crop-disease/Train.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ghana-crop-disease/Train.csv'

Exploration des données

Prétraitement des données

Modélisation

Entraînement du modèle

Évaluation du modèle